In [10]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Q1. Outlier가 데이터 셋에 존재할 경우 선형회귀에 대한 성능을 비교해보세요


# 데이터들의 의미
(https://www.kaggle.com/c/nyc-taxi-trip-duration/data)

id - a unique identifier for each trip

vendor_id - a code indicating the provider associated with the trip record

pickup_datetime - date and time when the meter was engaged

dropoff_datetime - date and time when the meter was disengaged

passenger_count - the number of passengers in the vehicle (driver entered value)

pickup_longitude - the longitude where the meter was engaged

pickup_latitude - the latitude where the meter was engaged

dropoff_longitude - the longitude where the meter was disengaged

dropoff_latitude - the latitude where the meter was disengaged

store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip



# target value: trip_duration - duration of the trip in seconds

In [ ]:
data_df = pd.read_csv('./train.csv')

In [ ]:
data_df.info()

모든 feature가 수치형 데이터임이 아님을 알 수있다.

id, vendor_id, store_and_fwd_flag에 대한 처리가 필요!

In [ ]:
np.round(data_df.describe(),2)

min, quantile, max를 살펴보면 어떤 outlier가 trip_duration에 있음을 알 수있습니다.

vendor id, id는 각각 identify를 위한 데이터이므로 drop을 사용하여 처리합니다. pick up time, drop off time은 각각 시계열 데이터인데, 이 책에서 시계열 처리에 대해 다루지 않았으므로 drop을 해줍니다.

In [5]:
#pandas의 drop 메서트를 활용하여 위의 feature를 drop해보세요

In [ ]:
data_df.head(3)

store_and_fwd_flag와 trip_duration의 상관관계를 알아본다. 만약 상관관계가 적다면 drop하고 그렇지 않다면 핸들링을 따로 한다

In [ ]:
sff_cat = {'N':0, 'Y':1}
cat_fun = lambda x:sff_cat[x]
sff = data_df['store_and_fwd_flag'].map(cat_fun)
data_df.groupby('store_and_fwd_flag')['trip_duration'].mean()

위의 결과를 보고 store_and_fwd_flag을 drop할지 하지 않을지 생각해보세요

trip_duration에 outlier 수를 파악해보고, 이 outlier들을 drop했을 때와 안했을 때의 성능 수준을 선형 회귀 모델들로 비교해 보겠습니다.

In [ ]:
np.sum(data_df.trip_duration > 10000)

# outlier가 있는 데이터셋을 사용하여 선형 회귀 모델 평가.

In [ ]:
#outlier를 제거하지 않은 데이터 셋을 훈련세트와 데이터 세트로 나눈다.
X = data_df.drop('trip_duration', axis = 1)
y = data_df.trip_duration

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

LinearRegression을 사용하여 train set을 학습시킨후 test set으로 평가해보세요

Pipeline과 PolynomialFeatures를 가지고 다항회귀를 사용해서 평가해보세요

Ridge 모델 학습

다양한 learning rate 가지고 ridge 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

Lasso 모델 학습

다양한 learning rate 가지고 Lasso 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

ElasticNet 모델 학습

다양한 learning rate 가지고 ElasticNet 모델을 평가해보세요.

이 때 각각의 coefficient를 확인해보세요

# outlier를 없엔 데이터 셋을 사용하여 선형 회귀모델 평가.

In [ ]:
outlier_idx = y[y > 10000].index
data_dropped_outlier = data_df.drop(outlier_idx, axis = 0)

In [ ]:
#데이터가 2000개 가량 줄어든 것을 확인할 수 있다.
data_dropped_outlier.info()

In [ ]:
X_dropped = data_dropped_outlier.drop('trip_duration', axis = 1)
y_dropped = data_dropped_outlier.trip_duration

X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_dropped, random_state = 42)

앞서서 했던 outlier를 제거하기 전 시행했던 각각의 모델들을 위의 데이터셋을 통해 평가해보세요.

# Q2. 이 데이터 셋에서는 규제가 그리 효과적이지 못한 것 처럼 보입니다. 그 이유는 무엇일까요?



# Q3. 이 데이터셋은 우리가 원하는 trip duration을 구하기에는 연관이 적은 것 같습니다. 데이터를 가공한다면, 어떤식으로 하면 trip duration과 관련이 있을까요?

# 부록:파이썬의 시계열 데이터 

파이썬 라이브러리에는 날짜와 시간을 위한 자료형과 달력과 관련된 기능을 제공하는 자료형이 있습니다..

파이썬 기본 자료형: datetime, time, calender

date: 그레고리언 달력을 사용하여 년, 월, 일을 저장한다.

time: 하루 중 시간을 시간, 분, 초, 마이크로 초 단위로 저장한다.

datetime: 날짜와 시간을 같이 저장한다

timedelta: 두 datetime 값 간의 차이(일, 초, 마이크로초)를 표현한다

In [ ]:
trip_df = pd.read_csv('./train.csv')
trip_df.drop(['id', 'vendor_id'], axis= 1,inplace = True)

간단한 시계열 포멧

%Y: 네자리 연도

%y: 두자리 연도

%m: 두자리 월 ex:01 은 1월

%d: 두자리 일 ex:03은 3일

%H: 24시간 형식 시간

%I: 12시간 형식 시간

%M: 두 자리 분

%S: 초

In [ ]:
from datetime import datetime
# series에 있는 시계열 데이터는 현재 문자열 형태로 저장되어 있기 때문에 처리가 용이한 datetime 타입으로 바꾼다.
d2dt = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S') #datetime형으로 변환할 때 시계열 데이터의 형태를 맞춰서 넣어줘야한다.
pickup = trip_df.pickup_datetime.map(d2dt)
dropoff = trip_df.dropoff_datetime.map(d2dt)

In [ ]:
#pickup 시간과 dropoff 시간의 간격을 초 단위로 바꾼다.
# datetime 형을 빼거나 더하면 timedelta 타입으로 바뀐다. timedelta 형은 시계열 형 데이터의 기간을 표현하는데 적합하다.
timegap = dropoff - pickup
tomsec = lambda x: x.seconds
timegap_sec = timegap.map(tosec)

In [ ]:
#timedelta형임을 확인할 수 있다.
type(timegap[0])

In [ ]:
timegap_sec

In [ ]:
#손님을 태운 시간과 내려준 시간의 차, 즉 trip_duration이 실제로는 이 데이터셋으로부터 바로 구할 수있다.
#위 데이터셋에서 trip_duration과 timegap_sec가 맞지 않는 4개의 데이터셋이 있는데 이는 시간이 잘못 기록되었음을 유추할 수 있다.
np.sum(timegap_sec != trip_df.trip_duration)